In [1]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub
pip install transformers datasets wandb accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=11bec6103937276b5d82780362eb438e94679b2dea06ea6c19aaab164efaae02
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install tf-keras

In [3]:
!pip install transformers datasets wandb accelerate

In [14]:
import wandb
from datasets import Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq


In [5]:
# Step 2: Set up WandB for Experiment Tracking
wandb.login()  # Login to WandB (you'll need an API key from your WandB account)
wandb.init(project="flan-t5-fine-tuning", name="flan-t5-qa")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: az-morinig (az-morinig7). Use `wandb login --relogin` to force relogin


Loading the FLAN-T5 Model


In [6]:
# Step 3: Load the Pre-Trained Model and Tokenizer
MODEL_NAME = "google/flan-t5-large"  # Using Flan-T5 Large model (500M+ parameters)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Testing - Before Fine tunning

In [7]:
test_inputs = [
    "Answer the following question: How old are you?",
    "Answer the question: How can I solve my Python code problem?",
    "Answer the question: How can I print a hello world in java?"
]

print("Testing the model before fine-tuning")
for input_text in test_inputs:
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True).input_ids
    outputs = model.generate(input_ids, max_length=50)
    print(f"Input: {input_text}")
    print(f"Output: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")

Testing the model before fine-tuning
Input: Answer the following question: How old are you?
Output: 18
Input: Answer the question: How can I solve my Python code problem?
Output: Use a recursive method.
Input: Answer the question: How can I print a hello world in java?
Output: Use a java.split() function to print the hello world.


In [8]:
# Step 4: Define or Load Dataset
# Option 1: Load a Hugging Face dataset (e.g., Yahoo Answers)
DATA_NAME = "yahoo_answers_qa"
stackoverflow_python = load_dataset(DATA_NAME, split='train[:1%]')  # Load a small subset for testing

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

yahoo_answers_qa.py:   0%|          | 0.00/2.76k [00:00<?, ?B/s]

The repository for yahoo_answers_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/yahoo_answers_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/87362 [00:00<?, ? examples/s]

In [9]:
stackoverflow_python

Dataset({
    features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
    num_rows: 874
})

In [10]:
#stackoverflow_python_qa = stackoverflow_python["train"].train_test_split(test_size=0.3)

In [11]:
def preprocess_function(examples):
    # Use the correct column names for Yahoo Answers dataset:
    inputs = examples["question"]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

 tokenization of the dataset



In [12]:
tokenized_dataset = stackoverflow_python.map(preprocess_function, batched=True)

Map:   0%|          | 0/874 [00:00<?, ? examples/s]

prepare data collator

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Fine- tunning the model

In [16]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-finetuned",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    fp16=True,
    report_to="wandb"                 #to track with WandB
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Define Trainer

In [17]:
trainer = Seq2SeqTrainer(
    model=model,                         # Pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=tokenized_dataset,     # Tokenized training dataset
    eval_dataset=tokenized_dataset,      # Tokenized evaluation dataset
    tokenizer=tokenizer,                 # Tokenizer
    data_collator=data_collator          # Handles padding dynamically
)


<ipython-input-17-12071f7deb15>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [20]:
import torch

# Explicitly set the device to CPU
device = "cpu"

# Move the model to CPU
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

Test the model before fine-tuning

In [21]:

print("Testing the model before fine-tuning...")
test_inputs = [
    "Answer the following question: How old are you?",
    "Answer the question: How can I solve my Python code problem?",
    "Answer the question: How can I print a hello world in java?"
]

for input_text in test_inputs:
    # Ensure the inputs are on the same device as the model (CPU)
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True).input_ids.to(device)

    # Generate outputs
    outputs = model.generate(input_ids, max_length=50)

    print(f"Input: {input_text}")
    print(f"Output: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")


Testing the model before fine-tuning...
Input: Answer the following question: How old are you?
Output: 18
Input: Answer the question: How can I solve my Python code problem?
Output: Use a recursive method.
Input: Answer the question: How can I print a hello world in java?
Output: Use a java.split() function to print the hello world.


In [22]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
0,0.000000,nan
1,0.000000,nan
2,0.000000,nan


TrainOutput(global_step=327, training_loss=0.0, metrics={'train_runtime': 532.7618, 'train_samples_per_second': 4.922, 'train_steps_per_second': 0.614, 'total_flos': 194833918844928.0, 'train_loss': 0.0, 'epoch': 2.9931350114416477})

In [23]:
#Save the Fine-Tuned Model
model.save_pretrained("./flan-t5-finetuned")
tokenizer.save_pretrained("./flan-t5-finetuned")

('./flan-t5-finetuned/tokenizer_config.json',
 './flan-t5-finetuned/special_tokens_map.json',
 './flan-t5-finetuned/spiece.model',
 './flan-t5-finetuned/added_tokens.json',
 './flan-t5-finetuned/tokenizer.json')

Testing the Model After Fine-Tuning

In [24]:
print("\nTesting the  model after fine-tuning")
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-t5-finetuned")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./flan-t5-finetuned")

for input_text in test_inputs:
    input_ids = fine_tuned_tokenizer(input_text, return_tensors="pt", truncation=True).input_ids
    outputs = fine_tuned_model.generate(input_ids, max_length=50)
    print(f"Input: {input_text}")
    print(f"Output: {fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)}")


Testing the  model after fine-tuning
Input: Answer the following question: How old are you?
Output: 18
Input: Answer the question: How can I solve my Python code problem?
Output: Use a recursive method.
Input: Answer the question: How can I print a hello world in java?
Output: Use a java.split() function to print the hello world.
